In [1]:
from zipfile import ZipFile
from io import BytesIO
import urllib.request as urllib2

import nltk 
nltk.download('punkt') #to tokenize sentences.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import nltk
nltk.download('punkt')

r = urllib2.urlopen("http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip").read()
file = ZipFile(BytesIO(r))
file.extractall()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# Load the data
lines = open('/content/cornell movie-dialogs corpus/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('/content/cornell movie-dialogs corpus/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [0]:
# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [0]:
# Create a list of all of the conversations' lines' ids.
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [7]:

print(id2line['L198'])
print(id2line['L199'])

You're asking me out.  That's so cute. What's your name again?
Forget it.


In [8]:
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []
for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

print(questions[:5])
print(answers[:5])

221616
221616
['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.', "Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part.  Please.', "You're asking me out.  That's so cute. What's your name again?", "No, no, it's my fault -- we didn't have a proper introduction ---"]
["Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part.  Please.', "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?", 'Forget it.', 'Cameron.']


In [0]:
#choosing number of training samples
num_samples = 10000  #Feel free to change anyone
questions_train = questions[:num_samples]
answers_train = answers[:num_samples]

In [0]:
#tokenization for each sentence
questions_tok = [nltk.word_tokenize(sent) for sent in questions]
#answers_tok = [nltk.word_tokenize(sent) for sent in answers]

In [0]:
full_tok = [item for sublist in questions_tok for item in sublist]
full_tok = [x.lower() for x in full_tok]
full_tok = [word for word in full_tok if word.isalpha() or word  in ['!','.','?',',']]

In [38]:
full_tok[:20]

['can',
 'we',
 'make',
 'this',
 'quick',
 '?',
 'roxanne',
 'korrine',
 'and',
 'andrew',
 'barrett',
 'are',
 'having',
 'an',
 'incredibly',
 'horrendous',
 'public',
 'up',
 'on',
 'the']

In [0]:
def train_ngram_lm(data, order=3):
    """
        Train n-gram language model
    """
    
    # pad (order-1) special tokens to the left
    # for the first token in the text
    order -= 1
    data = ['<S>'] * order + data #
    lm = defaultdict(Counter)
    
    # get ngrams for all sizes
    for k in range(1, order + 2):
      
      # loop through ngrams to get counts
      for i in range(len(data) - k + 1):
          """
          IMPLEMENT ME!

          """  
          # rolling window of ngrams
          ngrams = data[i:i + k]
          
          # split ngrams into previous word and next word
          next_word = ngrams.pop()
          
          # concatenate previous words with space
          ngrams = " ".join(ngrams)

          # add count of next word in ngram
          lm[ngrams].update([next_word])

    # convert Counter() object to dict
    lm = {key:  dict(values) for key, values in lm.items()}

    # normalize by counts
    for i in lm.keys():
      total = sum(lm[i].values(), 0.0)
      lm[i] = {k: v / total for k, v in lm[i].items()}
    return lm

In [0]:
import torch, pickle, os, sys, random, time
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn, optim
from collections import *
import numpy as np

In [0]:
# generate text
def generate_text(lm, vocab, context="he is the", order=3, num_tok=10):
    
    # The goal is to generate new words following the context
    # If context has more tokens than the order of lm, 
    # generate text that follows the last (order-1) tokens of the context
    # and store it in the variable `history`
    order -= 1
    history = context.split()[-order:]
    
    
    # `out` is the list of tokens of context
    # you need to append the generated tokens to this list
    out = context.split()
    punctuation = 0
    
    while punctuation < 2:
        """
        IMPLEMENT ME!
        
        """
        # if the context word has more tokens than the order of lm
        if len(history) > order:
          history = history[-(order - 1):]
          
        # for unigram models
        if order == 0:
          history = []
        
        # concatenate history with space
        context = " ".join(history)

        # look up context distribution
        dist = lm[context]
        
        # find next word from distribution
        next_word = list(np.random.choice(list(dist.keys()), 1, list(dist.values())))
        
        # only output the first two sentences
        if next_word[0] in ['.','!','?']:
          punctuation += 1
        
        # append next word to out
        out += next_word
       
        # update history
        history = history[1:]
        history += next_word
    
    return " ".join(out[3:])

In [0]:
order = 4
vocab = list(set(full_tok))
model = train_ngram_lm(full_tok, order=order)

In [67]:
generate_text(model, vocab, context='he is the', order=order)

['he', 'is', 'the']
courageous
captain
of
compliments
.
he
fights
as
you
sing
song
,
keeps
time
,
distance
,
and
proportion
he
rests
his
minim
rest
,
one
,
i
spare
no
expense
.


'courageous captain of compliments . he fights as you sing song , keeps time , distance , and proportion he rests his minim rest , one , i spare no expense .'

In [0]:
def chat_with_chatbot():
  firstrun = 0
  while True:
      if firstrun == 0:
        print("Hello my name is dumb chatbot! Let's chat but I may not make much sense. Type goodbye to exit.")
        firstrun +=1

      response = str(input())
      response_tok = nltk.word_tokenize(response)
      response_tok = [x.lower() for x in response_tok]
      response = " ".join(response_tok)
      if response == "goodbye":
          break
      else:
        try:
          print(generate_text(model, vocab, context=response, order=order))
        except:
          print("sorry, I couldn't understand that!")

In [79]:
chat_with_chatbot()

Hello my name is dumb chatbot! Let's chat but I may not make much sense. Type goodbye to exit.
How are you?
? nuts ? you bastard !
why are you nuts!?
sorry, I couldn't understand that!
why not?
how much for the tour of our humble plant . i must deactivate you .
That is mean!
sorry, I couldn't understand that!
I am angry.
sorry, I couldn't understand that!
why!
has , but ai i got melanie in hermosa beach . she tells me these answers , right place ?
no.
do . eating ?
goodbye
